# SNS Sentiment Analysis BERT

Analyze social media texts and measure potential inflammatory / offensive language with a BERT neural network.

(This does not work, unable to pass data through the BERT model, Iter nor Tensor conversions are working at all for the data and model).

In [75]:
import ast
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
from transformers import TFBertModel, BertTokenizer

In [2]:
# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased', output_hidden_states=False, return_dict=False)
bert_model.trainable = False

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

# Load the data

In [73]:
# Load dataset
train = pd.read_csv("./datasets/processed/train.csv")
X_train = train[train.columns[0]].apply(ast.literal_eval)[:800]
y_train = train[train.columns[1]].astype(int)[:800]
print(X_train.shape, y_train.shape)

test = pd.read_csv("./datasets/processed/test.csv")
X_test = test[test.columns[0]].apply(ast.literal_eval)[:200]
y_test = test[test.columns[1]].astype(int)[:200]
print(X_test.shape, y_test.shape)

(800,) (800,)
(200,) (200,)


In [76]:
print("Tensorflow Version : " ,tf.__version__)
print("Transformers Version : " ,transformers.__version__)

Tensorflow Version :  2.14.0
Transformers Version :  4.35.0


In [42]:
train_text = [" ".join(s) for s in X_train]
test_text = [" ".join(s) for s in X_test]

In [43]:
example_input = tokenizer(["Test sentence", "This is a test sentence"], padding=True, truncation=True, return_tensors='tf')
example_input

{'input_ids': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[ 101, 3231, 6251,  102,    0,    0,    0],
       [ 101, 2023, 2003, 1037, 3231, 6251,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [77]:
BATCH_SIZE = 4
MAX_LENGTH = 128

def data_generator(text, labels, batch_size: int) -> (dict, tf.Tensor):
    num_samples = len(text)
    i = 0
    print(len(text), len(labels))
    while True:
        x_data, y_data = [], []
        for j in range(batch_size):
            start = i * batch_size
            try:
                x_data.append(text[start + j])
                y_data.append(labels[start + j])
            except KeyError:
                print("INDEX:", j, "START:", start, "\n", labels)
            i += 1


        if i == num_samples // batch_size:
            i = 0
        
        batch_x = tokenizer(x_data, return_tensors="tf", max_length=MAX_LENGTH, padding='max_length', truncation=True)
        batch_y = tf.convert_to_tensor(y_data)

        yield dict(batch_x), batch_y

In [ ]:
print(np.shape(train_text), np.shape(y_train))
print(np.shape(test_text), np.shape(y_test))

In [ ]:
train_data_gen = data_generator(train_text, y_train, BATCH_SIZE)
test_data_gen = data_generator(test_text, y_test, BATCH_SIZE)

print(np.shape(next(train_data_gen)), np.shape(next(test_data_gen)))

# train_text_t = tokenizer(train_text, return_tensors="tf", max_length=MAX_LENGTH, padding='max_length', truncation=True)
# test_text_t = tokenizer(test_text, return_tensors="tf", max_length=MAX_LENGTH, padding='max_length', truncation=True)
# 
# train_dataset = tf.data.Dataset.from_tensor_slices((train_text_t['input_ids'], train_text_t['attention_mask'], y_train))
# test_dataset = tf.data.Dataset.from_tensor_slices((test_text_t['input_ids'], test_text_t['attention_mask'], y_test))
# 
# train_dataset = train_dataset.batch(BATCH_SIZE)
# test_dataset = test_dataset.batch(BATCH_SIZE)

In [58]:
class SNSBERTClassification(tf.keras.Model):
    def __init__(self, bert_model_, num_classes=2):
        super().__init__()
        self.bert = bert_model_
        self.fc = tf.keras.layers.Dense(num_classes, activation="sigmoid")

    def call(self, inputs, training=None, mask=None):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [59]:
bert_classifier = SNSBERTClassification(bert_model, num_classes=1)

bert_classifier.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
bert_history = bert_classifier.fit(
    train_data_gen,
    batch_size=BATCH_SIZE,
    steps_per_epoch=len(train_text) // BATCH_SIZE,
    validation_data=test_data_gen,
    validation_steps=BATCH_SIZE * 4,
    validation_batch_size=BATCH_SIZE,
    epochs=1
)

In [ ]:
model_ = tf.keras.Sequential([
    bert_model,
    tf.keras.layers.Lambda(lambda x: x[0][:,0,:]),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model_.fit(
    train_data_gen,
    batch_size=BATCH_SIZE,
    steps_per_epoch=len(train_text) // BATCH_SIZE,
    validation_data=test_data_gen,
    validation_steps=len(test_text) // BATCH_SIZE,
    validation_batch_size=BATCH_SIZE,
    epochs=3
)

In [ ]:
bert_classifier.evaluate_generator(test_data_gen)